In [167]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, model_selection,tree, linear_model, neighbors, naive_bayes, ensemble 
from matplotlib import pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from path import Path


## Visualizing the data and checking for correlations

In [168]:
# Load the viral_data.csv dataset.
filepath = Path('viral_data.csv')
viral_df = pd.read_csv(filepath, index_col=0)
print(viral_df.shape)
viral_df.head()

(200, 7)


,State,Year,total_pop,Uninsured_%,PC_Income,GDP,Flu_%
0,Alabama,2016.0,4863525.0,9.1,39536.0,191523,0.237400
1,Alaska,2016.0,741456.0,14.0,56302.0,53289,0.143367
2,Arizona,2016.0,6941072.0,10.0,40801.0,291259,0.322227
3,Arkansas,2016.0,2989918.0,7.9,40385.0,113490,0.096424
4,California,2016.0,39167117.0,7.3,58048.0,2519133,0.112745


In [169]:
# Generate our categorical variable list
viral_list = viral_df.dtypes[viral_df.dtypes == "object"].index.tolist()

In [170]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(viral_df[viral_list]))
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(viral_list)
encode_df.head()

,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,State_Colorado,State_Connecticut,State_Delaware,State_Florida,State_Georgia,...,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [171]:
# Merge one-hot encoded features and drop the originals
viral_df = viral_df.merge(encode_df,left_index=True, right_index=True)
viral_df = viral_df.drop(viral_list,1)
print(viral_df.shape)
viral_df.head()

(200, 56)


,Year,total_pop,Uninsured_%,PC_Income,GDP,Flu_%,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,...,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,2016.0,4863525.0,9.1,39536.0,191523,0.237400,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016.0,741456.0,14.0,56302.0,53289,0.143367,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016.0,6941072.0,10.0,40801.0,291259,0.322227,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016.0,2989918.0,7.9,40385.0,113490,0.096424,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016.0,39167117.0,7.3,58048.0,2519133,0.112745,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [172]:
viral_df.head()

,Year,total_pop,Uninsured_%,PC_Income,GDP,Flu_%,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,...,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,2016.0,4863525.0,9.1,39536.0,191523,0.237400,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016.0,741456.0,14.0,56302.0,53289,0.143367,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016.0,6941072.0,10.0,40801.0,291259,0.322227,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016.0,2989918.0,7.9,40385.0,113490,0.096424,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016.0,39167117.0,7.3,58048.0,2519133,0.112745,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [173]:
# Remove flu_% target from features data
y = np.array(viral_df['Flu_%'])
X = viral_df.drop('Flu_%', axis = 1)

In [174]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [175]:
# Check dat for correctness
print('Training Features Shape:', X_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (160, 55)
Testing Features Shape: (40, 55)
Training Labels Shape: (160,)
Testing Labels Shape: (40,)


In [176]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);

In [177]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
predictions

array([0.95212926, 0.12035331, 0.57534639, 0.33380226, 0.1480135 ,
       0.36635478, 0.10194584, 0.36098577, 0.35723435, 0.90167344,
       0.23871651, 0.33913137, 0.57195798, 0.10854649, 0.50976398,
       0.4537174 , 0.15961398, 0.34670707, 0.15942993, 0.15945133,
       0.21941737, 0.22627333, 0.18477445, 0.41146122, 0.20376258,
       0.1427818 , 0.09878894, 1.44865404, 0.4071206 , 0.22738401,
       0.58181765, 0.09688865, 0.19631073, 0.24094772, 0.62535958,
       0.21220535, 0.3053917 , 0.20514075, 0.20070386, 0.24197899])

In [178]:
# Claculating the root mean square
from sklearn.metrics import mean_squared_error
y_true = y_test
y_pred = predictions
mean_squared_error(y_true, y_pred, squared=False)

0.22031026433195453

In [179]:
# Calculate the r-squared
from sklearn.metrics import r2_score
r2_score(y_true, y_pred)

0.7063831256478982

In [180]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [181]:
# Create a random forest classifier.
rf_model = RandomForestRegressor(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)

In [182]:
# Calculating the root mean square
from sklearn.metrics import mean_squared_error
y_true = y_test
mean_squared_error(y_true, y_pred, squared=False)

0.21790218381740814

In [183]:
# Calculating the root square
from sklearn.metrics import r2_score
r2_score(y_true, y_pred)

0.7127667491037964